In [2]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

Implemention regression using sequential API of keras

In [13]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

In [14]:
model.compile(
    loss=keras.losses.mean_squared_error,
    optimizer="sgd"
)

In [16]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 4s 10ms/step - loss: 0.3948 - val_loss: 0.3792
Epoch 2/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3786 - val_loss: 0.3797
Epoch 3/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3795 - val_loss: 0.3741
Epoch 4/20
363/363 [==============================] - 3s 10ms/step - loss: 0.3707 - val_loss: 0.3695
Epoch 5/20
363/363 [==============================] - 3s 8ms/step - loss: 0.3767 - val_loss: 0.3679
Epoch 6/20
363/363 [==============================] - 3s 7ms/step - loss: 0.3671 - val_loss: 0.3722
Epoch 7/20
363/363 [==============================] - 3s 8ms/step - loss: 0.3667 - val_loss: 0.3700
Epoch 8/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3667 - val_loss: 0.3694
Epoch 9/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3624 - val_loss: 0.3651
Epoch 10/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3752 - val_loss: 0.36

In [17]:
model_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 1s 7ms/step - loss: 0.3839


In [18]:
X_new = X_test[:3]
y_pred = model.predict(X_new)

In [22]:
y_pred.round(3)

array([[1.221],
       [0.825],
       [2.884]], dtype=float32)

In [20]:
y_test[:3]

array([1.139, 0.736, 2.1  ])

**Functional** API using keras

In [23]:
input_= keras.layers.Input(shape = X_train.shape[1:]) 
hidden1 = keras.layers.Dense(30, activation ="relu")(input_) 
hidden2 = keras.layers.Dense(30, activation ="relu")(hidden1) 
concat = keras.layers.Concatenate()([input_, hidden2]) 
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

compile and fit will be same as above

**Multiinput** API's keras

But what if you want to send a subset of the features through the wide path and a different subset (possibly overlapping) through the deep path.
Example, suppose we have 5 fearues(0 to 4) through wide path and six features through(2 to 7) deep path

In [30]:
input_A = keras.layers.Input(shape=[5], name ="wide_input") 
input_B = keras.layers.Input(shape=[6], name ="deep_input") 
hidden1 = keras.layers.Dense(30, activation ="relu")(input_B) 
hidden2 = keras.layers.Dense(30, activation ="relu")(hidden1) 
concat = keras.layers.concatenate([input_A, hidden2]) 
output = keras.layers.Dense(1, name ="output")(concat)

model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [32]:
model.compile(loss=keras.losses.mean_squared_error, 
             optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [34]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:] 
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:] 
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[: 3], X_test_B[: 3]

In [35]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20, validation_data=((X_valid_A, X_valid_B), y_valid))

Epoch 1/20
363/363 [==============================] - 5s 13ms/step - loss: 2.5954 - val_loss: 0.9113
Epoch 2/20
363/363 [==============================] - 4s 10ms/step - loss: 0.7651 - val_loss: 0.6602
Epoch 3/20
363/363 [==============================] - 4s 10ms/step - loss: 0.6467 - val_loss: 0.6166
Epoch 4/20
363/363 [==============================] - 4s 10ms/step - loss: 0.6132 - val_loss: 0.5951
Epoch 5/20
363/363 [==============================] - 3s 9ms/step - loss: 0.5940 - val_loss: 0.5804
Epoch 6/20
363/363 [==============================] - 3s 8ms/step - loss: 0.5799 - val_loss: 0.5712
Epoch 7/20
363/363 [==============================] - 3s 9ms/step - loss: 0.5676 - val_loss: 0.5604
Epoch 8/20
363/363 [==============================] - 3s 8ms/step - loss: 0.5588 - val_loss: 0.5533
Epoch 9/20
363/363 [==============================] - 3s 8ms/step - loss: 0.5496 - val_loss: 0.5458
Epoch 10/20
363/363 [==============================] - 3s 9ms/step - loss: 0.5429 - val_loss: 0.

In [36]:
model.evaluate((X_test_A, X_test_B), y_test)

162/162 [==============================] - 1s 6ms/step - loss: 0.5168


0.516836404800415

In [37]:
y_pred = model.predict((X_new_A, X_new_B))

In [38]:
y_pred

array([[1.2890582],
       [1.6821191],
       [2.5410814]], dtype=float32)

In [39]:
y_test[:3]

array([1.139, 0.736, 2.1  ])

To make **multioutput** API's in keras

In [40]:
input_A = keras.layers.Input(shape=[5], name ="wide_input") 
input_B = keras.layers.Input(shape=[6], name ="deep_input") 
hidden1 = keras.layers.Dense(30, activation ="relu")(input_B) 
hidden2 = keras.layers.Dense(30, activation ="relu")(hidden1) 
concat = keras.layers.concatenate([input_A, hidden2]) 
output = keras.layers.Dense(1, name ="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)

model = keras.Model(inputs=[input_A, input_B], 
                   outputs=[output, aux_output])

In [41]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

**Observations**
here we need to specify loss for each of the output and weights are the importants given to each of the loss

In [42]:
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=20, 
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))

Epoch 1/20
363/363 [==============================] - 5s 12ms/step - loss: 1.0016 - main_output_loss: 0.9087 - aux_output_loss: 1.8374 - val_loss: 0.6514 - val_main_output_loss: 0.5883 - val_aux_output_loss: 1.2197
Epoch 2/20
363/363 [==============================] - 4s 11ms/step - loss: 0.6029 - main_output_loss: 0.5441 - aux_output_loss: 1.1329 - val_loss: 0.5556 - val_main_output_loss: 0.5027 - val_aux_output_loss: 1.0322
Epoch 3/20
363/363 [==============================] - 4s 11ms/step - loss: 0.5316 - main_output_loss: 0.4864 - aux_output_loss: 0.9384 - val_loss: 0.5083 - val_main_output_loss: 0.4677 - val_aux_output_loss: 0.8738
Epoch 4/20
363/363 [==============================] - 4s 11ms/step - loss: 0.5644 - main_output_loss: 0.5334 - aux_output_loss: 0.8436 - val_loss: 0.4928 - val_main_output_loss: 0.4598 - val_aux_output_loss: 0.7894
Epoch 5/20
363/363 [==============================] - 4s 10ms/step - loss: 0.4997 - main_output_loss: 0.4714 - aux_output_loss: 0.7549 - val

In [43]:
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))

162/162 [==============================] - 1s 6ms/step - loss: 0.4055 - main_output_loss: 0.3893 - aux_output_loss: 0.5512


In [44]:
total_loss, main_loss, aux_loss

(0.40551453828811646, 0.38933122158050537, 0.5511640906333923)

In [45]:
y_pred_main, y_pred_aux = model.predict((X_test_A, X_test_B))

**Subclassing API's**